In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from controller import controller
from nonlinear_dynamics import simulate
from mld_dynamics import mld, h
from visualizer import vis, animate

## Open-loop solution

In [ ]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

In [ ]:
# solve in open loop
# options = {'OptimalityTol': 1e-06, 'Presolve': 0, 'Heuristics': 0}
from time import time
tic = time()
solution, leaves, nodes, solver_time = controller.feedforward(
    x0,
    printing_period=None,
    gurobi_params={'OutputFlag': 0, 'Method': 1} # dual simplex
#     draw_label='Cart pole with walls'
)
print(time()-tic)
solver_time

In [ ]:
import cProfile
gurobi_params={'OutputFlag': 0, 'Method': 1}
cProfile.run('controller.feedforward(x0, printing_period=None, gurobi_params=gurobi_params)')

In [ ]:
# options = {'OutputFlag': 1, 'MIPGap': 0, 'Presolve': 0, 'Heuristics': 0}
options = {'OutputFlag': 0}
variables_gurobi, obj_gurobi, nodes_gurobi, solver_time = controller.feedforward_gurobi(x0, options)
solver_time

In [ ]:
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'], h)

## Solve with and without warm start

In [ ]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
x1 = solution.variables['x'][1]
warm_start, ws_time, interstep_time = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)
interstep_time

In [ ]:
import cProfile
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
cProfile.run('controller.construct_warm_start(leaves, x0, uc0, ub0, e0)')

In [ ]:
solution, leaves, nodes, solver_time = controller.feedforward(
    x1,
    warm_start=warm_start,
    gurobi_options={'Method': 1}, # dual simplex
#     draw_label='Cart pole with walls warm start'
)
solver_time

In [ ]:
import cProfile
gurobi_params={'OutputFlag': 0, 'Method': 1}
cProfile.run('controller.feedforward(x1, printing_period=None, warm_start=warm_start, gurobi_params=gurobi_params)')

In [ ]:
solution, leaves, nodes, solver_time = controller.feedforward(x1)
solver_time

In [ ]:
ub_shifted = controller.shift_binary_solution(variables_gurobi['ub'])
options = {'OutputFlag': 0}
variables_gurobi_1, obj_gurobi, nodes_gurobi, solver_time = controller.feedforward_gurobi(x1, options, ub_shifted)
solver_time